# REVIEWS
- `main_legs` and `start_legs` issue 
    - may need to generate the `start_legs` for every trips due to the shuttle trip issue of start_place
- Whether any minimum number of trips needed . Issue with shuttle trip 



### SET UP AND IMPORTS 

In [29]:
# SET UP AND IMPORTS 
import pandas as pd 
import datetime
import time


df = pd.read_excel('dataset/processed/psl.xlsx').reset_index(drop=True)
 
# Adding the Terminal Gap and Duty only for processing 
df['Terminal Gap'] = [0] * len(df)
df['Duty'] = [0] * len(df)

START_PLACE = 'PSL'
#start_legs contain the legs with departure place as start_place
start_legs = df[df['Departure Place'] == START_PLACE].sort_values('Departure Time', ascending=True)
start_legs.reset_index(drop=True,inplace=True)
#main_legs contains all other legs 
# main_legs = df[df['Departure Place'] != START_PLACE]
# MIN_DUTY = 27000 #setting the minimum duty as 7:00 hrs 
MIN_DUTY = 27000 #setting the minimum duty as 7:00 hrs 
# The ideal duty is 7:30 hrs with a 15 min break at the start and end . Then total 8:00 
# MAX_DUTY = 30600 # setting the maximum duty as 8:30 hrs
MAX_DUTY = 36000 # setting the maximum duty as 10:00 hrs
MIN_TERMINAL_GAP = 300
MAX_TERMINAL_GAP = 900
MAX_SPLIT = 1800 # Setting the maximum split time as 30 min 
BREAK_SPLIT = 1800 # break time of 30 min to avoid 5hrs continuous working

success = 0 # Not necessary , just to count the number of start legs taken 
trips_count = 0 # count the number of trips 
exe_generateTempSet  = 0
exe_popTempSet  = 0



### SUPPORTING FUNCTIONS 

#### Generate Tempset 
- Below function creates the temset for a particular leg in the stack . It checks for the needed criteria and then creates the set of legs that are possibly gives us a feasible solution 

In [30]:
# SUPPORTING FUNCTIONS
def generateTempSet(leg) -> pd.DataFrame:
    # FN LOGIC IS NOT CORRECT , NEED TO CREATE THE TEMP SET ONLY WITH THE CRITERIA
    # SORTING THEM AFTER CREATING THE ENTIRE TEMPSET IS NOT OPTIMAL
    global exe_generateTempSet
    x = time.time()

    # Creating a copy of the main legs
    temp_set_df = df[(df['Departure Place'] == leg['Arrival Place'])
                  & (df['Departure Time'] > leg['Arrival Time act'])]
    temp_set_df = temp_set_df.sort_values(
        'Departure Time', ascending=True).reset_index(drop=True)
    temp_set_df['Terminal Gap'] = temp_set_df['Departure Time'] - leg['Arrival Time act']
    # Sorting and resetting index
    # '''BELOW LINE MAY BE NOT NECESSARY'''
    # display(temp_set)
    temp_set = temp_set_df[
        (temp_set_df['Terminal Gap'] >= MIN_TERMINAL_GAP) & (
            temp_set_df['Terminal Gap'] < MAX_TERMINAL_GAP)  # |
        # (temp_set['Duty'] >= MIN_DUTY) & (temp_set['Duty'] <= MAX_DUTY)
    ]
# THERE ARE CHANCES THAT THE TEMP SET MAY BE EMPTY , SO WE ARE TRYING FOR 
#  THE NEXT LEVEL SEARCHING
    max_split = MAX_SPLIT
    ADD_ON = 900 # 15 min
    while len(temp_set) < 2 and max_split < 3600: # 1 hr split
        temp_set = temp_set_df[
            (temp_set_df['Terminal Gap'] >= MIN_TERMINAL_GAP) & (
                temp_set_df['Terminal Gap'] < max_split)  # |
            # (temp_set['Duty'] >= MIN_DUTY) & (temp_set['Duty'] <= MAX_DUTY)
        ]
        max_split += ADD_ON
    # temp_set['Duty'] = leg['Duty'] + temp_set['Running Time'] + temp_set['Terminal Gap']
    temp_set.loc[:, 'Duty'] = leg['Duty'] + temp_set['Running Time'] + temp_set['Terminal Gap']
    y = time.time()
    exe_generateTempSet += (y - x)
    # if len(temp_set) < 2 : print(f"Size of tempset == {len(temp_set)}")
    return temp_set

#### Pop Tempset 
- This function pops the currently visited leg from the corresponding tempset, then the tempset will always contain the possible legs from a given leg that satisfies the criteria needed   

In [31]:
def popTempSet(temp_set) -> pd.DataFrame:
    global exe_popTempSet
    x = time.time()

    temp_set.drop(0, inplace=True)
    # print("popTempSet")
    # display(temp_set)
    temp_set.reset_index(inplace=True)
    temp_set.drop('index',axis= 1, inplace=True)

    y = time.time()
    exe_popTempSet += (y - x)
    return temp_set


#### Display Trip
- Returns the trip once the stack is satisfying all the criteria 

In [38]:
def displayTrip(stack):
    global trips_count
    frame = pd.DataFrame()
    for x in stack:
        frame = pd.concat([frame, (x['current_leg'].to_frame()).T], ignore_index=True)
    frame['Terminal Gap'] = frame['Terminal Gap'].shift(-1)
    duty = second=frame['Duty'].iloc[len(frame) - 1]
    hrs, rem = divmod(duty, 3600)
    mint, sec = divmod(rem , 60)
    duty = datetime.time(hour=hrs, minute=mint, second= sec)
    frame = frame.drop('Duty', axis=1)

    trips_count += 1
    return [frame, duty]

def seconds_to_time(df, columns):
    df[columns] = df[columns].apply(lambda x: pd.to_datetime(x, unit='s').dt.strftime('%H:%M:%S'))


In [33]:
# check_df = pd.DataFrame(
#     {'Sl No.': [1, 124, 313, 641, 336, 695],
#      'Departure Time': [13200, 13800, 18300, 22500, 29400, 36300],
#      'Departure Place': ['PSL', 'KLKV', 'TVM', 'KNVLA', 'MC', 'KLKV'],
#      'Route of Operation': ['NH',
#                             'NH',
#                             'NH-UDA',
#                             'UDA',
#                             'NH',
#                             'PVR-VZM-BYPASS-TVM-NH-KLKV'],
#      'Arrival Place': ['KLKV', 'TVM', 'KNVLA', 'MC', 'KLKV', 'PSL'],
#      'Arrival Time act': [13500, 17700, 21900, 28800, 36000, 48900],
#      'KM': [3.5, 33.7, 32.0, 39.0, 40.0, 81.5],
#      'Running Time': [300, 3900, 3600, 6300, 6600, 12600],
#      'Terminal Gap': [300, 600, 600, 600, 300, None]}
# )



#### Remove Legs
- Once a leg is taken for a trip , it should be removed so that it will not be considered for the next round of iterations  

In [34]:
def removeLegs(trip) -> None:
    global success
    for i in range(len(trip)):
        if trip.iloc[i]['Departure Place'] == START_PLACE:
            # remove from both df and start_legs 
            df.drop((df[df['Sl No.'] == trip.iloc[i]['Sl No.']]).index, axis= 0, inplace=True)
            start_legs.drop((start_legs[start_legs['Sl No.'] == trip.iloc[i]['Sl No.']]).index, axis= 0, inplace=True)
            df.reset_index(drop=True, inplace=True)
            start_legs.reset_index(drop=True, inplace=True)
            success += 1
        else:
            # remove only from df 
            df.drop((df[df['Sl No.'] == trip.iloc[i]['Sl No.']]).index, axis= 0, inplace=True)
            df.reset_index(drop=True, inplace=True)
        



### ALGORITHM

In [35]:
# ALGORITHM
def generateTrip(start_leg):
    global error
    try:
        stack = [{"current_leg": start_leg, "temp_set": pd.DataFrame()}] # contain the legs in the trip 

        temp_set = generateTempSet(start_leg)
        stack.append({"current_leg": temp_set.iloc[0], "temp_set": popTempSet(temp_set)})

        while len(stack) > 1 : # and stack[-1]['current_leg']['Arrival Place'] != START_PLACE
            temp_set = generateTempSet(stack[-1]['current_leg'])
            
            if temp_set.empty :
                stack.pop()
                while(stack[-1]['temp_set'].empty) :
                        stack.pop()
                stack[-1]['current_leg'] = stack[-1]['temp_set'].iloc[0]
                stack[-1]['temp_set'] = popTempSet(stack[-1]['temp_set'])

                # display(displayTrip(stack))
                continue
            stack.append({"current_leg": temp_set.iloc[0], "temp_set": popTempSet(temp_set)})


            if (top_leg:=stack[-1]['current_leg'])['Arrival Place'] == START_PLACE:
                if MIN_DUTY > top_leg['Duty']:
                    # NEED TO ADDRESS SHUTTLE TRIP ISSUE OF START_PLACE
                    while(stack[-1]['temp_set'].empty) :
                        stack.pop()
                    stack[-1]['current_leg'] = stack[-1]['temp_set'].iloc[0]
                    stack[-1]['temp_set'] = popTempSet(stack[-1]['temp_set'])

                elif top_leg['Duty'] > MAX_DUTY:
                    stack.pop()
                    while(stack[-1]['temp_set'].empty) :
                        stack.pop()
                    stack[-1]['current_leg'] = stack[-1]['temp_set'].iloc[0]
                    stack[-1]['temp_set'] = popTempSet(stack[-1]['temp_set'])

                else: # MIN_DUTY <= top_leg['Duty'] <= MAX_DUTY 
                    #ideal case 
                    break;
        result = displayTrip(stack)
        removeLegs(result[0])
        display(result[0])

        display(result[0].to_dict(orient='list'))
        print("Steering Duty = ", result[1]) 
           
    except Exception as e:
            # removeLegs(start_leg)
            start_legs.drop(0, axis=0, inplace=True)
            start_legs.reset_index(drop=True, inplace=True)
            print(f"Error occured = {e}")
            return 

In [36]:
# # ALGORITHM
# def generateTrip(start_leg):
#         stack = [{"current_leg": start_leg, "temp_set": pd.DataFrame()}] # contain the legs in the trip 
        
#         temp_set = generateTempSet(start_leg)
#         print(temp_set)
#         stack.append({"current_leg": temp_set.iloc[0], "temp_set": popTempSet(temp_set)})

#         while len(stack) > 1 : # and stack[-1]['current_leg']['Arrival Place'] != START_PLACE
#             temp_set = generateTempSet(stack[-1]['current_leg'])
            
#             if temp_set.empty :
#                 stack.pop()
#                 while(stack[-1]['temp_set'].empty) :
#                         stack.pop()
#                 stack[-1]['current_leg'] = stack[-1]['temp_set'].iloc[0]
#                 stack[-1]['temp_set'] = popTempSet(stack[-1]['temp_set'])

#                 # display(displayTrip(stack))
#                 continue
#             stack.append({"current_leg": temp_set.iloc[0], "temp_set": popTempSet(temp_set)})


#             if (top_leg:=stack[-1]['current_leg'])['Arrival Place'] == START_PLACE:
#                 if MIN_DUTY > top_leg['Duty']:
#                     # NEED TO ADDRESS SHUTTLE TRIP ISSUE OF START_PLACE
#                     while(stack[-1]['temp_set'].empty) :
#                         stack.pop()
#                     stack[-1]['current_leg'] = stack[-1]['temp_set'].iloc[0]
#                     stack[-1]['temp_set'] = popTempSet(stack[-1]['temp_set'])

#                 elif top_leg['Duty'] > MAX_DUTY:
#                     stack.pop()
#                     while(stack[-1]['temp_set'].empty) :
#                         stack.pop()
#                     stack[-1]['current_leg'] = stack[-1]['temp_set'].iloc[0]
#                     stack[-1]['temp_set'] = popTempSet(stack[-1]['temp_set'])

#                 else: # MIN_DUTY <= top_leg['Duty'] <= MAX_DUTY 
#                     #ideal case 
#                     break;
#         result = displayTrip(stack)
#         removeLegs(result[0])
#         display(result[0])
#         print("Steering Duty = ", result[1]) 
           

### MAIN PART

In [37]:
# MAIN
x = time.time()

if __name__ == "__main__":
    # generateTrip(start_legs.iloc[0])
    # generateTrip(start_legs.iloc[1])
    # generateTrip(start_legs.iloc[2])
    while(not start_legs.empty):
        generateTrip(start_legs.iloc[0])
    print(f"Success legs = {success} ..... Missing {104 - success} routes  ")
    print(f"NUMBER OF TRIPS = {trips_count}")

y = time.time()
print(f"Time = {y - x}")
print(f"Time taken by tempset = {exe_generateTempSet}")
print(f"Time taken by popTempSet = {exe_popTempSet}")

,Sl No.,Departure Time,Departure Place,Route of Operation,Arrival Place,Arrival Time act,KM,Running Time,Terminal Gap
0,1,13200,PSL,NH,KLKV,13500,3.5,300,300
1,124,13800,KLKV,NH,TVM,17700,33.7,3900,600
2,313,18300,TVM,NH-UDA,KNVLA,21900,32.0,3600,600
3,641,22500,KNVLA,UDA,MC,28800,39.0,6300,600
4,336,29400,MC,NH,KLKV,36000,40.0,6600,300
5,695,36300,KLKV,PVR-VZM-BYPASS-TVM-NH-KLKV,PSL,48900,81.5,12600,None


{'Sl No.': [1, 124, 313, 641, 336, 695],
 'Departure Time': [13200, 13800, 18300, 22500, 29400, 36300],
 'Departure Place': ['PSL', 'KLKV', 'TVM', 'KNVLA', 'MC', 'KLKV'],
 'Route of Operation': ['NH',
  'NH',
  'NH-UDA',
  'UDA',
  'NH',
  'PVR-VZM-BYPASS-TVM-NH-KLKV'],
 'Arrival Place': ['KLKV', 'TVM', 'KNVLA', 'MC', 'KLKV', 'PSL'],
 'Arrival Time act': [13500, 17700, 21900, 28800, 36000, 48900],
 'KM': [3.5, 33.7, 32.0, 39.0, 40.0, 81.5],
 'Running Time': [300, 3900, 3600, 6300, 6600, 12600],
 'Terminal Gap': [300, 600, 600, 600, 300, None]}

Steering Duty =  09:50:00


KeyboardInterrupt: 

In [ ]:
df

,Sl No.,Departure Time,Departure Place,Route of Operation,Arrival Place,Arrival Time act,KM,Running Time,Terminal Gap,Duty
0,2,28500,PSL,NH,KLKV,29100,3.5,600,0,0
1,3,24000,PSL,NH,KLKV,24600,3.5,600,0,0
2,10,23400,PSL,NH,KLKV,24000,3.5,600,0,0
3,16,22200,PSL,NH,KLKV,22800,3.5,600,0,0
4,20,22800,PSL,NH,KLKV,23100,3.5,300,0,0
...,...,...,...,...,...,...,...,...,...,...
288,719,67200,MC,TVM-NTA-KLKV,PSL,73500,43.5,6300,0,0
289,720,49200,PSL,KLKV-KRKM,VLRD,52200,20.5,3000,0,0
290,721,54000,VLRD,PSL,TVM,60900,46.0,6900,0,0
291,725,38700,KLKV,KRKM,VLRD,41400,17.0,2700,0,0
